In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [52]:
data = pd.read_csv('Churn_Modelling.csv', delimiter=';')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [53]:
data = data.drop(['RowNumber','Surname','CreditScore'],axis=1)
data

,CustomerId,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,15606229,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,15569892,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,15584532,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,15682355,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [54]:
label_encoder_gen = LabelEncoder()
data['Gender'] = label_encoder_gen.fit_transform(data['Gender'])
data

,CustomerId,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,France,0,42,2,0.00,1,1,1,101348.88,1
1,15647311,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,15619304,France,0,42,8,159660.80,3,1,0,113931.57,1
3,15701354,France,0,39,1,0.00,2,0,0,93826.63,0
4,15737888,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,15606229,France,1,39,5,0.00,2,1,0,96270.64,0
9996,15569892,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,15584532,France,0,36,7,0.00,1,0,1,42085.58,1
9998,15682355,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [55]:
from sklearn.preprocessing import OneHotEncoder

OHE_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = OHE_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=OHE_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [56]:
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CustomerId,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,15634602,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,15647311,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,15619304,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,15701354,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,15737888,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [57]:
x = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [58]:
with open('label_encoder_gender.pkl','wb') as file : 
    pickle.dump(label_encoder_gen,file)

with open('One_Hot_Encoder_geo.pkl','wb') as file : 
    pickle.dump(OHE_geo,file)

with open('scaler.pkl','wb') as file : 
    pickle.dump(scaler,file)


### ANN regression

In [59]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
#import datetime

### Build Model

In [60]:
model = Sequential([
    Dense(64,activation='relu',input_shape=((X_train.shape[1],))), #first HL connected with input layer
    Dense(32,activation='relu'),                                   #HL 2
    Dense(1)                                                       #Output Layer
]
)

#compile
model.compile(optimizer='adam' ,loss='mean_absolute_error' ,metrics=['mean_squared_error'] )

C:\Users\SEMEH SAYADI\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [61]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [62]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime


log_dir="regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [63]:
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [64]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[tensorflow_callback,early_stopping_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 100052.7812 - mean_squared_error: 13307480064.0000 - val_loss: 98577.5156 - val_mean_squared_error: 13018887168.0000
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99581.9531 - mean_squared_error: 13262956544.0000 - val_loss: 97443.0078 - val_mean_squared_error: 12795414528.0000
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 97810.0234 - mean_squared_error: 12806573056.0000 - val_loss: 94515.0859 - val_mean_squared_error: 12217512960.0000
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 94777.9531 - mean_squared_error: 12295299072.0000 - val_loss: 89460.6875 - val_mean_squared_error: 11221021696.0000
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 88897.3828 - mean_squared_error: 11089661952.0000 - val_loss: 82616.2656 - val_mean_squared_error: 9868102656.0000
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 82876.5781 - mean_squared_error: 9851678720.0000 - val

In [65]:
%load_ext tensorboard

In [67]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6007 (pid 14120), started 0:00:36 ago. (Use '!kill 14120' to kill it.)

In [68]:
test_loss,test_mse = model.evaluate(X_test,y_test)
print(f'testMSE:{test_mse}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 51151.1758 - mean_squared_error: 3479511808.0000
testMSE:3406477056.0


In [70]:
model.save('regression_model.h5')